In [1]:
import time
import threading
import tkinter as tk
from PIL import Image, ImageTk  
from playsound import playsound
from tkinter import font, messagebox

In [2]:
class PomodoroApp:
    def __init__(self, root):
        #  🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅 🍅
        self.root = root
        self.root.title("My 🍅 Timer")
        self.is_running = False # flag for running the timer
        self.work_mins = 25 # default work duration in minutes
        self.break_mins = 5 # default break duration in minutes
        self.time_left = self.work_mins * 60 # convert to seconds
        self.is_working = True # flag for working timer and break timer
        self.cycles = 1 # default number of cycles
        self.cycle_number = 0 # current cycle number

        # Load the PNG background
        self.background_image = Image.open("background.png")  # Load your PNG file
        self.background_image = self.background_image.resize((400, 300), Image.LANCZOS)  # Resize to fit the window
        self.background_photo = ImageTk.PhotoImage(self.background_image)

        # Create a label for the background
        self.background_label = tk.Label(root, image=self.background_photo)
        self.background_label.place(relwidth=1, relheight=1)  # Fill the window

        # Fonts used
        self.font_text = font.Font(family="Angel wish", size=20, weight='bold', slant="italic")
        self.font_number = font.Font(family="Cloister Black", size=16, weight="normal")
        self.font_timer = font.Font(family="The Centurion", size=40, weight="bold", slant="italic")

        # Input for Work time
        self.work_label = tk.Label(root, text=' Work Time ( min ):', font=self.font_text, bg=self.root['bg'])
        self.work_label.pack(pady=1)
        self.work_entry = tk.Entry(root, font=self.font_number, bg=self.root['bg'])
        self.work_entry.pack(pady=1)
        self.work_entry.insert(0, str(self.work_mins)) # default value
        
        # Input for break time
        self.break_label = tk.Label(root, text=' Break Time ( min ):', font=self.font_text, bg=self.root['bg'])
        self.break_label.pack(pady=1)
        self.break_entry = tk.Entry(root, font=self.font_number)
        self.break_entry.pack(pady=1)
        self.break_entry.insert(0, str(self.break_mins)) # default value
        
        # Input for cycle
        self.cycle_label = tk.Label(root, text='Enter number of cycles:', font=self.font_text, bg=self.root['bg'])
        self.cycle_label.pack(pady=1)
        self.cycle_entry = tk.Entry(root, font=self.font_number)
        self.cycle_entry.pack(pady=1)
        self.cycle_entry.insert(0, str(self.cycles)) # default value
        
        # Timer display label
        self.timer_label = tk.Label(root, text=self.format_time(self.time_left), font=self.font_timer, bg=self.root['bg'])
        
        # Button frame
        self.button_frame = tk.Frame(root, bg=self.root['bg'], bd=0)
        self.button_frame.pack(side="bottom", fill='x', pady=10)

        # Start, Stop and Reset buttons
        self.start_button = tk.Button(self.button_frame, text='Start', bg=self.root['bg'], font=self.font_text, command=self.start_timer)
        self.start_button.pack(side='left', padx=10)
        
        self.stop_button = tk.Button(self.button_frame, text='Stop', bg=self.root['bg'], font=self.font_text, command=self.stop_timer)
        self.stop_button.pack(side='left', padx=10)
        
        self.reset_button = tk.Button(self.button_frame, text='Reset', bg=self.root['bg'], font=self.font_text, command=self.reset_timer)
        self.reset_button.pack(side='right', padx=10)
        
        # Set starting window size (width x height)
        self.root.geometry("")  # Width = 400, Height = 300

    def start_timer(self):
        # Get user input for the timers
        try:
            self.work_mins = float(self.work_entry.get())
            self.break_mins = float(self.break_entry.get())
            self.cycles = int(self.cycle_entry.get())
            self.time_left = self.work_mins * 60
            # Hide input fields
            self.work_label.pack_forget()
            self.work_entry.pack_forget()
            self.break_label.pack_forget()
            self.break_entry.pack_forget()
            self.cycle_label.pack_forget()
            self.cycle_entry.pack_forget()
            # Show timer label
            self.timer_label.pack(pady=10)
            self.root.geometry("")
            # Start running the timer
            self.is_running = True # set the running flag
            self.root.title(f"Work Timer 🍅") # Change title to work
            self.play_sound_threaded('SoulSteal.wav') # sound for start of work timer
            self.run_timer()
        except ValueError:
            messagebox.showerror("Invalid input", "Enter int or decimal for the timers and int for cycles, ex.: 1 - 0.5 - 1") 
        if self.work_mins <= 0 or self.break_mins <= 0 or self.cycles <= 0:
            messagebox.showerror("Invalid input", "Timers and cycles must be greater than zero.")
        return
    
    def run_timer(self):
        if  self.is_running:
            if self.time_left > 0:
                self.time_left -= 1
                self.timer_label['text'] = self.format_time(self.time_left)
                self.root.after(1000, self.run_timer) # call timer again after 1 second
            else:
                self.switch_timer()
    
    def switch_timer(self):
        if self.is_working:
            self.root.title(f"Break Timer 🍅") # change title to break
            self.time_left = self.break_mins * 60 # set timer for the break
            self.timer_label['text'] = self.format_time(self.time_left)
        else:
            self.root.title(f"Work Timer 🍅") # change title to work
            self.time_left = self.work_mins * 60 # set timer for work
            self.cycle_number += 1
            
        self.is_working = not self.is_working # change is_working True to False, to alternate cycles
        
        if self.cycle_number < self.cycles:
            if self.is_working:
                playsound('SoulSteal.wav') # play sound for work timer
            else:
                playsound('DarkMeta.wav') # play sound for break timer
            self.run_timer()
        else:
            self.play_sound_threaded('Impressive.wav')
            self.root.title(f"The end 🍅") # change title to work
            self.timer_label.config(text="Pomodoro Complete!", font=self.font_timer)
            self.root.geometry("")
            self.is_running = False
    
    def stop_timer(self):
        self.play_sound_threaded('What.wav') # sound for stop button
        self.is_running = False
    
    def reset_timer(self):
        self.play_sound_threaded('AsYouWish.wav')
        # Show input fields again
        self.work_label.pack()
        self.work_entry.pack()
        self.break_label.pack()
        self.break_entry.pack()
        self.cycle_label.pack()
        self.cycle_entry.pack()
        # Hide timer
        self.timer_label.pack_forget() 
        self.root.geometry("")
        self.is_running = False
        self.time_left = self.work_mins * 60 # reset to default
        self.cycle_number = 0 # reset to default
        self.timer_label['text'] = self.format_time(self.time_left)
       
    def format_time(self, time_in_seconds):
        minutes, seconds = divmod(time_in_seconds, 60)
        return f"🍅 {int(minutes):02}:{int(seconds):02} 🍅"

    def play_sound_threaded(self, sound_file):
        threading.Thread(target=playsound, args=(sound_file,)).start()


In [3]:
root = tk.Tk()
app = PomodoroApp(root)
root.mainloop()

In [1]:
from tkinter import Tk, font
root = Tk()
font.families()

('Academy Engraved LET',
 'Al Bayan',
 'Al Nile',
 'Al Tarikh',
 'American Typewriter',
 'Andale Mono',
 'Angel wish',
 'Apple Braille',
 'Apple Chancery',
 'Apple Color Emoji',
 'Apple SD Gothic Neo',
 'Apple Symbols',
 'AppleGothic',
 'AppleMyungjo',
 'Arial',
 'Arial Black',
 'Arial Hebrew',
 'Arial Hebrew Scholar',
 'Arial Narrow',
 'Arial Rounded MT Bold',
 'Arial Unicode MS',
 'Avenir',
 'Avenir Next',
 'Avenir Next Condensed',
 'Ayuthaya',
 'Baghdad',
 'Bangla MN',
 'Bangla Sangam MN',
 'Baskerville',
 'Beirut',
 'Big Caslon',
 'Bodoni 72',
 'Bodoni 72 Oldstyle',
 'Bodoni 72 Smallcaps',
 'Bodoni Ornaments',
 'Bradley Hand',
 'Brush Script MT',
 'Chalkboard',
 'Chalkboard SE',
 'Chalkduster',
 'Charter',
 'Cloister Black',
 'Cochin',
 'Comic Sans MS',
 'Copperplate',
 'Corsiva Hebrew',
 'Courier',
 'Courier New',
 'Damascus',
 'DecoType Naskh',
 'Devanagari MT',
 'Devanagari Sangam MN',
 'Didot',
 'DIN Alternate',
 'DIN Condensed',
 'Diwan Kufi',
 'Diwan Thuluth',
 'Euphemia UCAS